## Q.21. ヒストグラム正規化

#### Histogram normalization

ヒストグラム正規化を実装せよ。

ヒストグラムは偏りを持っていることが伺える。
例えば、0に近い画素が多ければ画像は全体的に暗く、255に近い画素が多ければ画像は明るくなる。
ヒストグラムが局所的に偏っていることを**ダイナミックレンジが狭い**などと表現する。
そのため画像を人の目に見やすくするために、ヒストグラムを正規化したり平坦化したりなどの処理が必要である。

このヒストグラム正規化は**濃度階調変換(gray-scale transformation)** と呼ばれ、[c,d]の画素値を持つ画像を[a,b]のレンジに変換する場合は次式で実現できる。
今回は*imori_dark.jpg*を[0, 255]のレンジにそれぞれ変換する。

<img src="assets/hist_norm_equ.png" width="400">

<!--
```bash
xout = {  a                         (xin < c)
         (b-a)/(d-c) * (xin-c) + a  (c <= xin <= d)
          b                         (d < xin)
```
-->

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("imori_dark.jpg").astype(np.float32)

a = 0
b = 255

c = img.min()
d = img.max()

out = (b-a)/(d-c)*(img-c)+a
out[img<c] = a
out[img>d] = b

plt.hist(out.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.savefig("training_IMG/training_21_hist.png")
plt.show()

cv2.imwrite("training_IMG/training_21.png", out)

In [ ]:
%reset -f

## Q.22. ヒストグラム操作

ヒストグラムの平均値をm0=128、標準偏差をs0=52になるように操作せよ。

これはヒストグラムのダイナミックレンジを変更するのではなく、ヒストグラムを平坦に変更する操作である。

平均値m、標準偏差s、のヒストグラムを平均値m0, 標準偏差s0に変更するには、次式によって変換する。

<img src="assets/hist_mani_equ.png" width="200">

<!--
```bash
xout = s0 / s * (xin - m) + m0
```
-->


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("imori_dark.jpg").astype(np.float32)

m0 = 128
s0 = 52

m = np.mean(img)
s = np.std(img)

out = s0/s*(img-m)+m0
out[out<0] = 0
out[out>255] = 255
out = out.astype(np.uint8)

plt.hist(out.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.savefig("training_IMG/training_22_hist.png")
plt.show()

cv2.imwrite("training_IMG/training_22.png", out)

In [ ]:
%reset -f

## Q.23. ヒストグラム平坦化

#### Histogram equalization

ヒストグラム平坦化を実装せよ。

ヒストグラム平坦化とはヒストグラムを平坦に変更する操作であり、上記の平均値や標準偏差などを必要とせず、ヒストグラム値を均衡にする操作である。

これは次式で定義される。
ただし、S ... 画素値の総数、Zmax ... 画素値の最大値、h(z) ... 濃度zの度数

<img src="assets/hist_equ_equ.png" width="200">

<!--
```bash
Z' = Zmax / S * Sum{i=0:z} h(z)
```
-->


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("imori_dark.jpg").astype(np.float32)
H, W, C = img.shape
S = H * W * C * 1.
z_max = 255
sum_h = 0.

out = img.copy()
for i in range(1, 255):
    ind = np.where(img == i) # 同じ値のインデックスを抽出
    sum_h += len(img[ind]) # 同じ値が何個あるか確認 = これが度数
    z_prime = z_max / S * sum_h
    out[ind] = z_prime

out = out.astype(np.uint8)

plt.hist(out.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.savefig("training_IMG/training_23_hist.png")
plt.show()

cv2.imwrite("training_IMG/training_23.png", out)

In [ ]:
%reset -f

## Q.24. ガンマ補正

#### Gamma correction

*imori_gamma.jpg*に対してガンマ補正(c=1, g=2.2)を実行せよ。

ガンマ補正とは、カメラなどの媒体の経由によって画素値が非線形的に変換された場合の補正である。
ディスプレイなどで画像をそのまま表示すると画面が暗くなってしまうため、RGBの値を予め大きくすることで、ディスプレイの特性を排除した画像表示を行うことがガンマ補正の目的である。

非線形変換は次式で起こるとされる。
ただしxは[0,1]に正規化されている。
c ... 定数、g ... ガンマ特性(通常は2.2)

<img src="assets/gamma_equ1.png" width="150">

<!--
```bash
x' = c * Iin ^ g
```
-->

そこで、ガンマ補正は次式で行われる。

<img src="assets/gamma_equ2.png" width="150">

<!--
```bash
Iout = (1/c * Iin) ^ (1/g)
```
-->

In [ ]:
import cv2
import numpy as np

img = cv2.imread("imori_dark.jpg").astype(np.float32)

c = 1
g = 2.2

out = img/255.

out = (1/c*out)**(1/g)
out = out.astype(np.uint8)

cv2.imwrite("training_IMG/training_24.png", out)

In [ ]:
%reset -f

## Q.25. 最近傍補間

最近傍補間により画像を1.5倍に拡大せよ。

最近傍補間(Nearest Neighbor)は画像の拡大時に最近傍にある画素をそのまま使う手法である。
シンプルで処理速度が速いが、画質の劣化は著しい。

次式で補間される。
I' ... 拡大後の画像、 I ... 拡大前の画像、a ... 拡大率、[ ] ... 四捨五入

<img src="assets/nni_fig.png" width="700">

<!--
```bash
I'(x,y) = I([x/a], [y/a])
```
-->


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("imori.jpg").astype(np.float32)

def nn_interpolate(img, ax=1, ay=1):
	H, W, C = img.shape

	# 拡大後のサイズを確認
	aH = int(ay*H)
	aW = int(ax*W)

	y = np.arange(aH).repeat(aW).reshape(aH, -1) # aH行の2次元配列を作成
	x = np.tile(np.arange(aW), (aH, 1)) # np.arange(aW)を(aH, 1)の形状分まで生成。aH行aW列の2次元配列を作る
	y = np.round(y/ay).astype(np.int8)
	x = np.round(x/ax).astype(np.int8)

	out = img[y,x]

	out = out.astype(np.uint8)

	return out

out = nn_interpolate(img, ax=1.5, ay=1.5)
cv2.imwrite("training_IMG/training_25.png", out)

In [ ]:
%reset -f